

<p align="center">
  <img src="Pictures/stackstorm.jpg">
  
</p>

# StackStorm 101

## LAB 1 The Basics
StackStorm is an automation framework that uses Sensors to watch and listen to alarm/event queues. Triggers to tell Rules when to launch, Actions/Workflows to change the existing state.

<p align="center">
  <img src="Pictures/stackstorm-view.png">
  
</p>

Stackstorm Doc [here](https://stackstorm.com/)

We will use the ssh kernel to remotely connect to an existing stackstorm server. 


## 1 - The concepts:

Let's start by connecting to the stackstorm server

In [ ]:
%login {{ STACKSTORMSRVNAME }}

A quick check of the StackStorm (st2) version.

In [ ]:
st2 --version

Getting help with st2

In [ ]:
st2 -h

Stackstorm is a platform that you will need to sign in to. Use the installation password.

In [ ]:
st2 login student{{  STDID  }} --password {{ PASSSTU  }}

Actions are the workhorse of stackstorm. From the diagram above they are the last thing that gets called to kick off some form of automation. To see what actions you have installed on your st2 server run this.

In [ ]:
st2 action list

Thats quite a few packs and actions. We can narrow down our search using the -p flag and specifying the pack we are interested in.

In [ ]:
st2 action list -p core

Additionally, you can look at sensors, rules, and triggers using the same syntax. Results will vary depending on what is installed on the st2 server.

In [ ]:
st2 sensor list

In [ ]:
st2 rule list

In [ ]:
st2 trigger list

You can also have st2 run commands either locally or remotely. As you noted from above, st2 has CORE actions that make this easy. This means any command you can run from the command
line can be executed in st2, either from the local host or any remote host. You will see the results of the command returned in stdout (standard out) and the command itself was successful.

In [ ]:
st2 run core.local -- date -R

Notice the id number? Everytime st2 executes a task, it tracks the outcome in the executions list.
You can view that list by running the below command.

In [ ]:
st2 execution list

Let us now isolate an id

In [ ]:
st2 execution list -j | jq ".[-1].id" | sed 's/"//g' > id

In [ ]:
st2 execution get $(cat id)

## 2 - Try on your own

Every one of the ’id’s’ will have all the the errors, stdin, stdout, parameters and return code. This is super handy when trying to debug any actions you are creating. You would use the st2 execution get (id) command. Since these numbers are randomly generated you will have to do this command on your own. It’s a bit tricky to copy and paste text in a notebook. Don’t worry about it now, just know you can always use st2 execution get to see the results of an execution.

# 2-1  Note:

When manually running an action from the st2 command line, you need to include the pack name, where the action lives, and the name of the action. st2 run pack.action. Actions may or may
not require extra parameters to function. To learn what any action requires, we need to look at the internals of the action by using the get command. Let’s look at the core pack and what is required for the echo action. Not all parameters are required, look carefully. If the parameter has a default setting, you don’t have to include it.

In [ ]:
st2 action get core.echo

Looks like we need to add the parameter message to our call for action. So we add message=’our string’.

In [ ]:
st2 run core.echo message='Lorem ipsum, blah, blah, blah'

OK, now let’s look at how we will run something a little more useful. Let’s list all the files in the directory.

In [ ]:
st2 run core.local cmd='ls'

Maybe you want to scrape some information from a web server, we can use this!

In [ ]:
st2 run core.http url="http://www.techworldwookie.com/test.html"

# 2-2  Custom Actions:

Let’s run a custom action. nasa_apod is an action that will get the NASA picture of the day for NASA’s server and return some information about the picture and the URL. This action is part of a stackstorm integration pack called tektalk. Let’s see if it is installed.

In [ ]:
st2 pack list

Yes, we see the pack tektalk listed, now let’s see what actions are made available to us.

In [ ]:
st2 action list -p tektalk

Great, now we have to see what paramerters are needed for the action to run.

In [ ]:
st2 action get tektalk.nasa_apod

API_KEY and HD are defaulted. The only parameter we need to supply is the date for the picture we want. It will need to be in a yyyy-mm-dd format.

## 2-3 Action Files

Actions are made up of two different files. An introductory YAML file and a script of some kind. Action files live in the /actions directory. Let’s take a look.

In [ ]:
ls -lia /opt/stackstorm/packs/tektalk/actions

Let’s look at the introductory YAML file

In [ ]:
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod.yaml

This is a YAML file so it will start with three dashes. We have the name of the action and the name of the pack it resides. The obilgatory description and then the runner_type. There are
13 different types of runners including WinRM Powershell. In our example we will be using a python script, nasa_apod.py. The file then informs the python script that it will be receiveing
three parameters. Let’s look at the python script.

In [ ]:
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod.py

We pass the api_key, date and hd variable to the python script and use requests to get the response. The python script returns the variable data. Let’s run the action like this!

In [ ]:
st2 run tektalk.nasa_apod date=2018-07-04

## HIP Tip of the Week!

It is very easy to take an existing shell script or python script and convert it to be used with stackstorm. You do not need to rewrite all of your automation scripts.

## 2-4 Workflows

Actions by themselves are basically just more of the same. They are just scripts. Workflows allow us to string several actions together and feed the output of one action into another. Wrokflows, like
actions require an intrductory YAML file and then another YAML file in the workflows directory. Workflows can call on any action from any st2 pack. The new workflow system for st2 is Orquesta,
let look at our example workflow. First the introductory YAML.

In [ ]:
cat /opt/stackstorm/packs/tektalk/actions/nasa_apod_workflow.yaml

The key takeaway is the runner_type is now Orquesta. Notice that the entry__point is another YAML with the exact same name located in the workflows directory. Let’s look at that file.

In [ ]:
cat /opt/stackstorm/packs/tektalk/actions/workflows/nasa_apod_workflow.yaml

Let’s break this down. This is just a list of tasks. Tasks are carried out by actions. Actions get parameters (or not). 
Results are published to the context where they can be reused by other actions.

# 3  The Elephant in the room

What we see here is not python. It is YAML (Yet Another Markup Language). I just takes a few lines of these YAML files to stich together a very complex workflows. Calling any actions from any pack is quite powerful. Using stackstorm will save you from writing thousands of lines of python.
You would not normally run a workflow manually, that’s really not automation becasue a human is involved. Workflows are assigned to st2 rules that get activated by triggers. That’s the magic of st2. In just this one special case however, we will manually run our workflow and see what goes on behind the scenes.

In [ ]:
st2 run tektalk.nasa_apod_workflow

So what happened? Our workflow had three tasks, get a picture from a webserver, echo the results, and tell us what linux we are ruinning. St2 informs us that all three tasks were executed successfully.
If you are curious about any of these executions, remember st2 lets us look at each one in greater detail with copy and pasting the execution id in the following command. To copy and paste
text, highlight the text, hold down the shift key and right-click just past the last character you have highlighted. Then, CTRL-v to paste! Easy, right?

Let's now isolate the first id

In [ ]:
st2 run tektalk.nasa_apod_workflow -j | jq  ".[0].id" 2>/dev/null | sed 's/"//g' > id

In [ ]:
st2 execution get $(cat id)

# 4 The stackstorm exchange

You can find over 170 prebuilt st2 integration packs on the echange. Let’s have a look. (Opens in a new browser!)
[StackStorm Exchange](https://exchange.stackstorm.org/)
You might of seen this on the Exchange website. Installing a pack in st2 is very easy. st2 pack install . Maybe you want to automate something in VMware. You would have to write a bunch
of python to make that happen, pyVmomi (a module written in python) would be your new best friend. Even so, you would still write a couple 1000 lines of code vs. st2 pack install vsphere.
You may have noticed that there is a pack for Ansible as well.This means you can use stackstorm to add event based automation to you Ansible deployment!

# 5 The stackstorm GUI

I promised you this from the beginning. It’s handy, I find I use the GUI more to monitor the sensors, rules and actions. After I get it put together from the command line (creating YAML files is way easier with an editor), I make sure it’s running properly from the GUI (So much easier than on the command line).

[The Jedi Link](https://{{ JPHOSTEXT }}:{{  STACKSTORMWEBUIPORT  }})

</br>
 <a href="0-ReadMeFirst.ipynb" target="New" title="Back: Introduction"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Conclusion.ipynb#sc" target="New" title="Next:Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-Conclusion.ipynb#sc" target="New" title="Next: Conclusion"><i class="fas fa-chevron-circle-right" style="color:#FFAD33;"></i></a></h2>